In [26]:
import numpy as np
import itertools as it
import functools

In [27]:
def norm_2(vec):
    return sum(el**2 for el in vec)

In [28]:
def get_order(order_template, m, n):
    return np.array(order_template).reshape((m, n))

def get_order_template(order):
    return tuple(order.flatten().data)

def find(arr, num):
    return tuple(el[0] for el in np.where(arr == num))

def swap_cols(arr, i, j):
    arr[:, [i, j]] = arr[:, [j, i]]
    
def swap_rows(arr, i, j):
    arr[[i, j], :] = arr[[j, i], :]

def get_min_rep(order):
    cpy = order.copy()
    # move 1 into first coordinate
    i, j = find(cpy, 1)
    swap_rows(cpy, 0, i)
    swap_cols(cpy, 0, j)
    # sort rest of columns by the first row
    cpy = cpy[:, np.argsort(cpy[0])]
    # sort rest of the rows by the first entry
    return cpy[np.argsort(cpy[:,0])]

In [29]:
def np_hash(arr):
    m = len(arr)
    n = len(arr[0])
    arr_num = sum(arr[i][j] * 2 ** (i * n + j) for i in range(m) for j in range(n))
    return (m, n, arr_num)

def np_dehash(hsh):
    m, n, arr_num = hsh
    return np.array([[(arr_num >> (i * n + j)) % 2 for j in range(n)] for i in range(m)])

In [30]:
def Q_calc(cart):
    c = cart.sum()
    m = len(cart)
    n = len(cart[0])
    v = np.array([sum(cart[i]) / c for i in range(m)])
    h = np.array([sum(cart.T[i]) / c for i in range(n)])
    return (norm_2(v - np.ones(m) / m) * m + norm_2(h - np.ones(n) / n) * n) / (m + n)

@functools.lru_cache(maxsize=None)
def Q(hsh):
    cart = np_dehash(hsh)
    return Q_calc(cart)

In [31]:
def get_Q_list(order_template, m, n):
    Q_list = []
    cart = np.ones((m, n), dtype=np.int)
    for i in range(1, m * n):
        r, c = divmod(order_template.index(i), n)
        cart[r][c] = 0
        Q_list.append(Q(np_hash(cart)))
    return Q_list

In [32]:
def get_min_order_reps(m, n):
    past = None
    for perm in it.permutations(range(1, m * n + 1)):
        if past is None or past < get_order_template(get_min_rep(get_order(perm, m, n))):
            past = perm
            yield perm
    return None

In [33]:
def get_equiv_reps(m, n):
    ret = np.zeros(m * n, dtype=np.int).reshape(m, n)
    ret[0][0] = 1
    for comb in it.combinations(range(2, m * n + 1), n - 1):
        nums = list(range(2, m * n + 1))
        ret[0,1:n] = comb
        for el in comb:
            nums.remove(el)
        for comb2 in it.combinations(nums, m - 1):
            nums2 = nums.copy()
            for el in comb2:
                nums2.remove(el)
            ret[1:,0] = comb2
            for rest in it.permutations(nums2):
                ret[1:,1:] = np.array(rest).reshape(m - 1, n - 1)
                yield tuple(ret.flatten().data)
    return None

In [34]:
def find_min_Q_sum(m, n, start=np.inf):
    min_orders = []
    min_Q_sum = start
    for order_template in get_equiv_reps(m, n):
        Q_sum = 0
        cart = np.ones((m, n), dtype=np.int)
        for i in range(1, m * n):
            r, c = divmod(order_template.index(i), n)
            cart[r][c] = 0
            Q_sum += Q(np_hash(cart))
            if Q_sum > min_Q_sum:
                break
        else:
            if Q_sum < min_Q_sum:
                min_Q_sum = Q_sum
                min_orders = [order_template]
                print(order_template, min_Q_sum, end="\r")
            elif Q_sum == min_Q_sum:
                min_orders.append(order_template)
    return min_Q_sum, min_orders, len(min_orders)

In [10]:
sum(get_Q_list((1, 3, 5, 7, 9, 11, 8, 10, 12, 2, 4, 6), 2, 6))

1.2939504572107388

In [ ]:
find_min_Q_sum(4, 4)